# DNN Retrieval Model

#### Two-Tower Model for Learning Query and Candidate Embeddings

References: 
- https://www.tensorflow.org/recommenders/examples/basic_retrieval
- https://storage.googleapis.com/pub-tools-public-publication-data/pdf/6c8a86c981a62b0126a11896b7f6ae0dae4c3566.pdf
_____

In this notebook, we will build a Two-Tower DNN Model system to recommend a set of courses from an online catalogue that a given user is likely to watch. One of the advantages of building a Deep Learning Recommendation engine is the ability to build rich, flexible feature representations.

## DNN: Retrival and Rankng Architecture 

![](../docs/assets/deep_retrieval.png) 

### I. Retrieval 

Stage is responsible for selecting an initial set of hundreds of candidates from all candidates and efficiently filter out candidates that a user is not interested in. Because the retrieval model could consists of millions of users and items, it has to be computationally efficient. We'll discuss more below on optimizing this stage, but first there are two main components of the retrieval model:

**1. Query Model**: computes the query representation (normally a fixed-dimensionality embedding vector) using query features. A query fearture could consist of, but not limited to the the following:

- unique id
- title
- previous_history
- content viewing time
- likes
- ratings
- view date

**2. Candidate model** computes the candidate representation (an equally-sized vector) using the candidate features. A candidate fearture could consist of:

- uqniue id
- title


The process in the retrieval stage is to translate User and Item ids into embedding vectors, which are just high-dimensional numerical representations. The weights for the embedding layers are adjuste during training and the outputs of the two models are then multiplied together to give a query-candidate affinity score, with higher scores expressing a better match between the candidate and the query.

### II. Ranking

Stage takes the outputs of the retrieval model and fine-tunes them to a much smaller subset of recommendations. For the ranking stage, one of the main differences with this architecture is the ability to substantially improve the recommendations by using more features rather than just user and candidate identifiers. We can include features about the candidate such as:

- prices
- genres
- posted time

_______
Keep in mind that **Matrix Factorization** `user_id` and `candidate_id` features collaboratively to learn the latent features and does not consider side-features like mentioned above, and thus may not be highly performant. We can use deep learnning architectures such as the two-tower nerual network to include side features into the model as shown below.

![](../docs/assets/two_tower_model.png) 

![](../docs/assets/two_tower_model2.png) 

________
### Evaluate Model

The training data contains positive (user, item) pairs. To evaluate the mode, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates. So if the score for the positive pair is higher than for all other candidates, our model is highly accurate. The metric we will be using to evaluate the model is:

- **Metrics:** The metric utilized for evaluating the model is [factorized_top_k.TopK](https://www.tensorflow.org/recommenders/api_docs/python/tfrs/metrics/FactorizedTopK), which computes the top K categorical accuracy: how often the true candidate is in the top K candidates for a given query.

    - As the model trains, the top-k retrieval metrics updates. The `factorized_top_k` retrieval metric measures the number of true positive that is in the top-k retrieved items from the entire candidate set. As an example, a top-5 categorical accuracy metric of 0.2 would tell us that, on average, the true positive is in the top 5 retrieved items 20% of the time.

        - **Recall:** the ratio of items that a user likes were actually recommended. If a user likes say 5 items and the recommendation shows 3 of them, then the recall is 0.60.

        - **Precision:** out of all the recommended items, how many the user actually liked? If 5 items were recommended to the user out of which he liked, say 4 of them, then the precision is 0.80.

         - So we if we recommend all items to a user, then we have 100% recall! If we recommend say 1000 items and the user only likes 10, the precision is 0.10%, which is very low. 

        - **Goal:** Maximize both the precision and recall. 

- **Loss Function**: The loss function [tfrs.tasks.Retrieval](https://www.tensorflow.org/recommenders/api_docs/python/tfrs/tasks/Retrieval) will try to maximize the affinity of these query, candidate pairs while minimizing the affinity between the query and candidates belonging to other queries in the batch.



In [1]:
%cd ../

/Users/tracesmith/Desktop/Trace/Coding/user-recommender


In [3]:
%load_ext autoreload
import os
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

from recommenders.retrieval import retrieval_main, QueryModel, CandidateModel
from recommenders.utils.plots import plot_metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Build Dataset

For this example, the dataset we will focus on is `user_course_views.csv`.

In [4]:
# User content
user_course_views = pd.read_csv(os.path.join('data','user_course_views.csv'))
course_tags = pd.read_csv(os.path.join('data','course_tags.csv'))

In [5]:
user_course_views['course_id'] = user_course_views['course_id'].apply(lambda x: ' '.join(x.split('-')))
courses = user_course_views[['course_id','author_handle','level']].drop_duplicates()

In [6]:
# Users --> Convert to Tensors
user_id = tf.convert_to_tensor(user_course_views['user_handle'].astype(str), dtype=tf.string)
users_watched_courses = tf.convert_to_tensor(user_course_views['course_id'].astype(str), dtype=tf.string)
users_watched_courses_view_time = tf.convert_to_tensor(user_course_views['view_time_seconds'].astype(str), dtype=tf.int64)
tensors = {'user_id':user_id,'title':users_watched_courses,'user_view_time':users_watched_courses_view_time}
users = tf.data.Dataset.from_tensor_slices(tensors)

2021-07-29 20:01:51.256160: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Courses --> Convert to Tensors
course_ids = tf.convert_to_tensor(courses['course_id'].astype(str), dtype=tf.string)
tensors = {'title':course_ids}
courses = tf.data.Dataset.from_tensor_slices(tensors)
courses = courses.map(lambda x: x['title'])

### Train Model

In [ ]:
%autoreload
model,history = retrieval_main(users,
                       courses,
                       epochs=10,
                       learning_rate=0.10,
                       batch_size=1_000,
                       split_ratio=0.80,
                       top_k=10)

### Model Prediction

- To answer a query with this approach, the system must first map the query to the embedding space.

- Here we perform an exhaustive search of the neighbors in an embedding vector. This means iterating over all possible items and computing the distance for each one of them to our query point.

In [64]:
tfrs.layers.factorized_top_k.BruteForce(model.query_model,k=5)
brute_force.index(courses.batch(1_0000).map(model.candidate_model),courses)

### Save Model

In [49]:
with tempfile.TemporaryDirectory() as tmp_dir:
    retrieval_model_path = os.path.join(tmp_dir, "retrieval_model")
brute_force.save('models')

/var/folders/ky/49p8hmz15hxchr2k1kvm6thm0000gn/T/tmpvky54kzc


### Evaluate Model

In [ ]:
%autoreload
plot_metrics(history,['factorized_top_k/top_5_categorical_accuracy','factorized_top_k/top_10_categorical_accuracy',
                      'factorized_top_k/top_50_categorical_accuracy','factorized_top_k/top_100_categorical_accuracy'])

### Model Summary

In [17]:
model.query_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
query_model_2 (QueryModel)   multiple                  280355    
_________________________________________________________________
dense_4 (Dense)              multiple                  1088      
Total params: 281,443
Trainable params: 281,440
Non-trainable params: 3
_________________________________________________________________


In [18]:
model.candidate_model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
candidate_model_2 (Candidate (None, 64)                222176    
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
Total params: 224,256
Trainable params: 224,256
Non-trainable params: 0
_________________________________________________________________


### Recommendations

To make a prediction, a query model takes in the features of the query and transforming them into a query embedding, and a candidate model. 

- As mentioned, we will be using [tfrs.layers.factorized_top_k.BruteForce](https://www.tensorflow.org/recommenders/api_docs/python/tfrs/layers/factorized_top_k/BruteForce), but this this approach is not suitable for large scale datasets since this layer sorts the pre-computed candidate representations, and calculate the scores of the query-candidate pairs for all possible candidates, and then returns the highest `k` ranked itmes. 

- An alternative approach is using approximate nearest neighbours (ANN) index which allows fast approximate lookup of candidates in response to a query produced by the query model. [ScaNN (Scalable Nearest Neighbors)](https://eugeneyan.com/writing/how-to-install-scann-on-mac/) is a method for efficient vector similarity search at scale.

- https://github.com/google-research/google-research/tree/master/scann

In [ ]:
brute_force = tf.keras.models.load_model('models')

In [103]:
brute_force(tf.constant(['1']))[1].numpy().ravel()

array([b'using schedules material cost estimation revit 1368',
       b'walk cycles motionbuilder 184',
       b'nuke green screen keying building fundamentals',
       b'learning program better programmer',
       b'learning program better programmer'], dtype=object)

### Serve Model

In [ ]:
!docker run -p 8501:8501 -v "/models" -e MODEL_NAME=retrieval -t tensorflow/serving &